In [62]:
# Import dependencies
import pandas as pd
import requests
from config import api_key

In [63]:
# Movies after processing
movie_list = pd.read_csv('resources/cleaned/movie_list_cleaned.csv', index_col = False)

In [64]:
movie_list.head()

,genre_ids,id,overview,popularity,release_date,title,vote_average,vote_count,keyword,est_date,date
0,"[18, 36, 35]",31561,The aging Caesar finds himself intrigued by th...,8.546,12/11/1945,Caesar and Cleopatra,6.3,36,['Egypt'],[-3150],-46
1,"[18, 10749, 12]",29993,When strongman Samson rejects the love of the ...,22.539,12/21/1949,Samson and Delilah,6.5,127,['Samson'],[-1118],-1100
2,"[18, 10749, 36]",42567,King David enters into an adulterous affair wi...,6.823,8/10/1951,David and Bathsheba,6.4,35,['Bathsheba'],[-975],-1000
3,"[36, 12]",114771,Cleopatra (Rhonda Fleming) toys with Mark Anto...,1.960,5/8/1953,Serpent of the Nile,7.7,3,"['Egypt', 'Rome', 'Mark Antony', 'Cleopatra']","[-3150, -753, -83, -51]",-41
4,"[36, 12]",350806,The Jews are taken from Jerusalem and made sla...,1.241,10/21/1953,Slaves of Babylon,6.0,1,"['Jerusalem', 'Cyrus', 'Persia', 'Persian']","[-2800, -550, -550, -550]",-580


In [65]:
movie_list.dtypes

genre_ids        object
id                int64
overview         object
popularity      float64
release_date     object
title            object
vote_average    float64
vote_count        int64
keyword          object
est_date         object
date              int64
dtype: object

In [66]:
movie_list = movie_list.sort_values('date')
movie_list.head()

,genre_ids,id,overview,popularity,release_date,title,vote_average,vote_count,keyword,est_date,date
53,"[12, 18]",62204,A colossal adventure odyssey that turns back t...,10.358,12/16/1981,Quest for Fire,7.1,354,['pre-historic'],[-10000],-100000
49,[],338039,"In the last Ice Age, a family of Cro-Magnons b...",0.600,12/11/1974,The Tribe,5.0,2,['Neanderthals'],[-15000],-50000
59,[18],96428,The biblical tale of Joseph is told from an Eg...,0.961,1/1/1994,The Emigrant,5.5,4,['Egypt'],[-3150],-1650
5,"[36, 18]",24973,"In eighteenth-dynasty Egypt, Sinuhe, a poor or...",8.218,8/25/1954,The Egyptian,6.1,54,"['Egypt', 'Pharoah']","[-3150, -3150]",-1400
23,"[18, 36]",77971,Chronicles the rise and fall of the woman who ...,3.184,9/20/1961,"Nefertiti, Queen of the Nile",5.0,11,['Nefertiti'],[-1370],-1340


In [67]:
# Create list of movie ids that will be used to make API calls for additional information

id_list = movie_list.id.tolist()
print(id_list)

[62204, 338039, 96428, 24973, 77971, 119913, 81409, 47446, 211139, 533220, 2719, 6844, 652, 372519, 493416, 5040, 90198, 29993, 298509, 206004, 42567, 58430, 29996, 153779, 60915, 366585, 565179, 138519, 223168, 350806, 330390, 164302, 18696, 1271, 7278, 19972, 199153, 263681, 53182, 168408, 15005, 119925, 53039, 286405, 178184, 383695, 74778, 138797, 42668, 458191, 228867, 43020, 68906, 113743, 169818, 273868, 330395, 207234, 205031, 284016, 208835, 31561, 112424, 326088, 330627, 465616, 613370, 8095, 533575, 5066, 509606, 81908, 114771, 270673, 106385, 168479, 49398, 321637, 58933, 58441]


In [68]:
# Code from The-Final-Project_F-PALS, Popular_Crew.ipynb, refractoring for this project.
# The code below will make an API call based on the movie list to get the leading actors and directors of each film.

famous_actors = pd.DataFrame()
directors = pd.DataFrame()
id_for_movie = []
director_movie_id = []

for movie_id in id_list:
    movie = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}&language=en-US').json()
    for actor in movie['cast']:
        famous_actors = famous_actors.append(actor, ignore_index=True)
        id_for_movie.append(movie_id)
    
    for director in movie['crew']:
        if director['job'] == "Director":
            directors = directors.append(director, ignore_index=True)
            director_movie_id.append(movie_id)

famous_actors["movie_id"] = id_for_movie
directors['movie_id'] = director_movie_id
famous_actors = famous_actors.rename(columns = {"id": "actor_id",'popularity': 'actor_popularity'})
famous_actors_cleaned = famous_actors[['name','actor_id','gender','character','actor_popularity','movie_id']]
directors['movie_id'] = director_movie_id
directors = directors.rename(columns={'id': 'director_id', 'popularity': 'director_popularity'})
director_clean = directors[['name','director_id','gender','director_popularity','movie_id']]


In [69]:
studio = pd.DataFrame()
id_for_movie = []

for movie_id in id_list:
    movieDetail = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US').json()
    for movie in movieDetail['production_companies']:
        studio = studio.append(movie, ignore_index=True)
        id_for_movie.append(id)

studio['movie_id'] = id_for_movie
studio= studio.drop(columns=['logo_path'])
studio = studio.rename(columns={'id': 'studio_id', 'name':'studio_name'})

In [70]:
# Add csv files to cleaned folder
famous_actors_cleaned.to_csv('resources/cleaned/actors_cleaned.csv', index=False)
director_clean.to_csv('resources/cleaned/director_cleaned.csv', index=False)
studio.to_csv('resources/cleaned/studio_cleaned.csv', index=False)

In [71]:
genre = requests.get(f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=en-US').json()['genres']
genre_df = pd.DataFrame(genre)
genre_df

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


In [72]:
requests.get(f'https://api.themoviedb.org/3/movie/7840?api_key={api_key}&language=en-US').json()

{'adult': False,
 'backdrop_path': '/ifDzpMogZmSjvqa8eMbRyiOcX5F.jpg',
 'belongs_to_collection': None,
 'budget': 105000000,
 'genres': [{'id': 12, 'name': 'Adventure'},
  {'id': 28, 'name': 'Action'},
  {'id': 18, 'name': 'Drama'},
  {'id': 14, 'name': 'Fantasy'}],
 'homepage': '',
 'id': 7840,
 'imdb_id': 'tt0443649',
 'original_language': 'en',
 'original_title': '10,000 BC',
 'overview': "A prehistoric epic that follows a young mammoth hunter's journey through uncharted territory to secure the future of his tribe.",
 'popularity': 25.434,
 'poster_path': '/lg3HcDLC5Kx3RNPQjeXvvS6nen3.jpg',
 'production_companies': [{'id': 347,
   'logo_path': None,
   'name': 'Centropolis Entertainment',
   'origin_country': ''},
  {'id': 923,
   'logo_path': '/5UQsZrfbfG2dYJbx8DxfoTr2Bvu.png',
   'name': 'Legendary Pictures',
   'origin_country': 'US'},
  {'id': 10884,
   'logo_path': None,
   'name': 'The Department of Trade and Industry of South Africa',
   'origin_country': ''},
  {'id': 9137,
